In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [2]:
from pathlib import Path
from tensorflow import keras
import os

# See where your process is running from
print("CWD:", os.getcwd())

# Point to your dataset root (adjust the name if different)
DATA_ROOT = Path("dogs_vs_cats")  # or "." if train/ is directly here
TRAIN_DIR = DATA_ROOT / "train"
VAL_DIR   = DATA_ROOT / "test"    # or "val" / "validation"

print("Train dir:", TRAIN_DIR.resolve())
print("Val dir:  ", VAL_DIR.resolve())
print("Train exists?", TRAIN_DIR.exists())
print("Val exists?  ", VAL_DIR.exists())

# Optional: show immediate subfolders (should be class names)
if TRAIN_DIR.exists():
    print("Classes under train/:", [p.name for p in TRAIN_DIR.iterdir() if p.is_dir()])

CWD: /Users/niravpolara/PycharmProjects/AI-Learning-Journey/12 Deep Learning/CNN/Project/DOGVSCAT
Train dir: /Users/niravpolara/PycharmProjects/AI-Learning-Journey/12 Deep Learning/CNN/Project/DOGVSCAT/dogs_vs_cats/train
Val dir:   /Users/niravpolara/PycharmProjects/AI-Learning-Journey/12 Deep Learning/CNN/Project/DOGVSCAT/dogs_vs_cats/test
Train exists? True
Val exists?   True
Classes under train/: ['dogs', 'cats']


In [3]:
from tensorflow import keras
from pathlib import Path

DATA_ROOT = Path("dogs_vs_cats")  # relative to your CWD
TRAIN_DIR = DATA_ROOT / "train"
VAL_DIR   = DATA_ROOT / "test"

img_size = (256, 256)
batch_size = 32

train_ds = keras.utils.image_dataset_from_directory(
    directory=TRAIN_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory=VAL_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=False,
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [4]:
#normlize
def process(image,label):
  image = tf.cast(image/255,tf.float32)
  return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [5]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape= (256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/Users/niravpolara/PycharmProjects/AI-Learning-Journey/venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,681 (56.64 MB)

 Trainable params: 14,847,489 (56.64 MB)

 Non-trainable params: 192 (768.00 B)

In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
 18/625 ━━━━━━━━━━━━━━━━━━━━ 14:00 1s/step - accuracy: 0.5694 - loss: 6.4508

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
import cv2

In [ ]:
test_img = cv2.imread(DATA_ROOT / "cat.jpg")

In [ ]:
test_img

In [ ]:
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img,(256,256))

In [ ]:
test_input = test_img.reshape((1,256,256,3))

In [ ]:
model.predict(test_input)

In [ ]:
test_img = cv2.imread(DATA_ROOT / 'dog.jpg' )

In [ ]:
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img,(256,256))

In [ ]:
test_input = test_img.reshape((1,256,256,3))

In [ ]:
model.predict(test_input)